<img src="NotebookAddons/blackboard-banner.png" width="100%" />

# Preparing a HyP3 InSAR Stack for MintPy

**Author**: Alex Lewandowski; University of Alaska Fairbanks
 
Based on [prep_hyp3_for_mintpy.ipynb](https://github.com/ASFHyP3/hyp3-docs) by Jiang Zhu; University of Alaska Fairbanks

<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />

**Important Note about JupyterHub**
Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## 0. Importing Relevant Python Packages

In this notebook we will use the following scientific libraries:
- [GDAL](https://www.gdal.org/) is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.

**Our first step is to import gdal and other needed packages**

In [ ]:
from osgeo import gdal
from pathlib import Path
import re

from osgeo import gdal
import pyproj

import asf_notebook as asfn

from hyp3_sdk import Batch, HyP3

%matplotlib notebook

## 1. Load Your Own Data Stack Into the Notebook

This notebook assumes that you've created an InSAR data stack over an area of interest using the [Alaska Satellite Facility's](https://www.asf.alaska.edu/) value-added product system HyP3, available via [ASF Data Search/Vertex](https://search.asf.alaska.edu/). HyP3 is an ASF service used to prototype value added products and provide them to users to collect feedback.

We will retrieve HyP3 data via the hyp3_sdk. As both HyP3 and the Notebook environment sit in the [Amazon Web Services (AWS)](https://aws.amazon.com/) cloud, data transfer is quick and cost effective.

Before downloading anything, create an analysis directory to hold your data.

**Select or create a working directory for the analysis:**

In [ ]:
while True:
    data_dir = Path(asfn.input_path(f"\nPlease enter the name of a directory in which to store your data for this analysis."))
    if data_dir.is_dir():
        contents = data_dir.glob('*')
        if len(list(contents)) > 0:
            choice = asfn.handle_old_data(data_dir, list(contents))
            if choice == 1:
                shutil.rmtree(data_dir)
                data_dir.mkdir()
                break
            elif choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        data_dir.mkdir()
        break

**Define absolute path to  analysis directory:**

In [ ]:
analysis_directory = Path.cwd().joinpath(data_dir)
print(f"analysis_directory: {analysis_directory}")

**Create a HyP3 object and authenticate**

In [ ]:
hyp3 = HyP3(prompt=True)

**List projects containing active InSAR products and select one:**

Your HyP3 InSAR project should include:
- DEMs
- Incidence Angle Maps

*DEMs and Inc Angle Maps are available as options when submitting a HyP3 project*

In [ ]:
my_hyp3_info = hyp3.my_info()
active_projects = dict()

for project in my_hyp3_info['job_names']:
    batch = Batch()
    batch = hyp3.find_jobs(name=project, job_type='INSAR_GAMMA').filter_jobs(running=False, include_expired=False)
    if len(batch) > 0:
        active_projects.update({batch.jobs[0].name: batch})
        
if len(active_projects) > 0:
    display(Markdown("<text style='color:darkred;'>Note: After selecting a project, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:darkred;'>Otherwise, you will rerun this code cell.</text>"))
    print('\nSelect a Project:')
    project_select = asfn.select_parameter(active_projects)

project_select

**Select a date range of products to download:**

In [ ]:
jobs = project_select.value

display(Markdown("<text style='color:darkred;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
dates = asfn.get_job_dates(jobs)
date_picker = asfn.gui_date_picker(dates)
date_picker

**Save the selected date range and remove products falling outside of it:**

In [ ]:
date_range = asfn.get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")
jobs = asfn.filter_jobs_by_date(jobs, date_range)

**Gather the available paths and orbit directions for the remaining products:**

In [ ]:
display(Markdown("<text style='color:darkred;'><text style='font-size:150%;'>This may take some time for projects containing many jobs...</text></text>"))
jobs = asfn.get_paths_orbits(jobs)
paths = set()
orbit_directions = set()
for p in jobs:
    paths.add(p.path)
    orbit_directions.add(p.orbit_direction)
display(Markdown(f"<text style=color:blue><text style='font-size:175%;'>Done.</text></text>"))

---
**Select a path:**

This notebook does not currently support merging InSAR products in multiple paths

In [ ]:
display(Markdown("<text style='color:darkred;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Path:')
path_choice = asfn.select_parameter(paths)
path_choice

**Save the selected flight path/s:**

In [ ]:
flight_path = path_choice.value
if flight_path:
    if flight_path:
        print(f"Flight Path: {flight_path}")
    else:
        print('Flight Path: All Paths')
else:
    print("WARNING: You must select a flight path in the previous cell, then rerun this cell.")

**Select an orbit direction:**

In [ ]:
if len(orbit_directions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a flight direction, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Flight Direction:')
direction_choice = asfn.select_parameter(orbit_directions, 'Direction:')
direction_choice

**Save the selected orbit direction:**

In [ ]:
direction = direction_choice.value
print(f"Orbit Direction: {direction}")

**Filter jobs by path and orbit direction:**

In [ ]:
jobs = asfn.filter_jobs_by_path(jobs, [flight_path])
jobs = asfn.filter_jobs_by_orbit(jobs, direction)
print(f"There are {len(jobs)} products to download.")

**Download the products, unzip them into a directory named after the product type, and delete the zip files:**

In [ ]:
print(f"\nProject: {jobs.jobs[0].name}")
project_zips = jobs.download_files(analysis_directory)
for z in project_zips:
    asfn.asf_unzip(str(analysis_directory), str(z))
    z.unlink()

# 2. Confirm Presence of DEMS and Incidence Angle Maps

These are optional addon products in HyP3, which are necessary for MintPy

In [ ]:
dems = list(analysis_directory.glob('*/*dem.tif'))
inc_maps = list(analysis_directory.glob('*/*inc_map.tif'))

if len(dems) > 0 and len(inc_maps) == len(jobs):
    print("Success: Found at least 1 DEM and an incidence angle map for every job.")
else:
    raise FileNotFoundError("Failed to find at least 1 DEM and an incidence angle map for every job. \
    \nYou will not be able to successfully run a MintPy time-series unless your reorder your HyP3 project \
with DEMS and Incidence Angle Maps included.")

# 3. Subset the Stack and Cleanup Unused Files

**Subset the stack to the largest aoi covered by all input files and delete unneeded files:**

In [ ]:
fnames = list(analysis_directory.glob('*/*.tif'))
fnames.sort()

# determine the largest area covered by all input files
corners = [gdal.Info(str(f), format='json')['cornerCoordinates'] for f in fnames]
ulx = max(corner['upperLeft'][0] for corner in corners)
uly = min(corner['upperLeft'][1] for corner in corners)
lrx = min(corner['lowerRight'][0] for corner in corners)
lry = max(corner['lowerRight'][1] for corner in corners)

# clip the files
# regex = '(?<=_)(corr|unw_phase|water_mask|dem|lv_theta|inc_map|inc_map_ell|amp)(?=.tif)'
regex = '.tif'
for fname in fnames:
    if re.search(regex, str(fname)):
        fname_out = fname.parent.joinpath(Path(f"{fname.stem}_clip{fname.suffix}"))
        gdal.Translate(destName=str(fname_out), srcDS=str(fname), projWin=[ulx, uly, lrx, lry])
        fname.unlink()
        
# remove the *.xml, png, and *.kmz files
for pattern in ["xml","png","kmz","md.txt"]:
    unneeded_files = analysis_directory.glob(f"*/*.{pattern}")
    for file in unneeded_files:
        file.unlink()

**Subset the stack to a smaller AOI or skip the rest of this section and proceed to section 4 for a link to the MintPy_Time_Series_From_Prepared_Data_Stack notebook**

In [ ]:
image_file = f"{analysis_directory}/raster_stack.vrt"
amp = list(analysis_directory.glob('*/*_amp_clip.tif'))[0]
!gdalbuildvrt -separate $image_file -overwrite $amp
img = gdal.Open(image_file)
rasterstack = img.ReadAsArray()

fig_xsize = 7.5
fig_ysize = 7.5
aoi = asfn.AOI_Selector(rasterstack, fig_xsize, fig_ysize)

In [ ]:
geotrans = img.GetGeoTransform()

def geolocation(x, y, geotrans):
    return [geotrans[0]+x*geotrans[1], geotrans[3]+y*geotrans[5]]

In [ ]:
try:
    ul = geolocation(aoi.x1, aoi.y1, geotrans)
    lr = geolocation(aoi.x2, aoi.y2, geotrans)
#     print(f"aoi_coords in EPSG {utm}")
    print(f"upper left corner: {ul}")
    print(f"lower right corner: {lr}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error may occur if an AOI was not selected.</text>'))
    display(Markdown(f'<text style=color:red>Note that the square tool icon in the AOI selector menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))

In [ ]:
clips = list(analysis_directory.glob('*/*clip.tif'))
clips.sort()

for i, fname in enumerate(clips):
    fname.rename(fnames[i])
    gdal.Translate(destName=str(fname), srcDS=str(fnames[i]), projWin=[ul[0], ul[1], lr[0], lr[1]])
    fnames[i].unlink() 

# 4. Proceed to the MintPy Time-Series Notebook

**Run the code cell below for a link to the MintPy_Time_Series_From_Prepared_Data_Stack notebook**

In [ ]:
display(Markdown(f'<text style=color:green> Open <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/notebooks/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb" target="_blank"> MintPy_Time_Series_From_Prepared_Data_Stack.ipynb </a>to run an InSAR time-series analysis on your data using MintPy</text>'))

<font face="Calibri" size="2"> <i>Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb - Version 1.1.0 - September 2021
    <ul>
        <li>Add DEM and Incidence Angle Map check</li>
        <li>Add AOI subsetting</li>
        <li>Link to MintPy_Time_Series_From_Prepared_Data_Stack.ipynb</li>
    </ul>
    </i>
</font>